# ModelChecker Jupyter Integration Demo

This notebook demonstrates the new Jupyter integration for the ModelChecker framework.

## 1. Environment Setup

First, we'll set up the environment to ensure the model_checker package is properly available.

In [ ]:
# Set up the environment
import sys
import os
import importlib

# Helper function to find the ModelChecker root directory
def find_modelchecker_root():
    # Start with current directory
    current_dir = os.getcwd()
    print(f"Current directory: {current_dir}")
    
    # Check if we're already in the project root or src directory
    if os.path.exists(os.path.join(current_dir, 'src', 'model_checker')):
        return current_dir
    elif os.path.exists(os.path.join(current_dir, 'model_checker')):
        return os.path.dirname(current_dir)
    
    # Try going up a few directories
    for levels_up in range(1, 6):  # Check up to 5 levels up
        parent = current_dir
        for _ in range(levels_up):
            parent = os.path.dirname(parent)
        if os.path.exists(os.path.join(parent, 'src', 'model_checker')):
            return parent
    
    # Check common installation paths
    common_paths = [
        os.path.expanduser("~/Documents/Philosophy/Projects/ModelChecker/Code"),
        os.path.expanduser("~/ModelChecker/Code")
    ]
    
    for path in common_paths:
        if os.path.exists(path) and os.path.exists(os.path.join(path, 'src', 'model_checker')):
            return path
    
    # Couldn't find it
    return None

# Find and set up project root
project_root = find_modelchecker_root()

if project_root:
    print(f"Found ModelChecker at: {project_root}")
    
    # Add paths to Python path
    paths_to_add = [
        project_root,
        os.path.join(project_root, 'src')
    ]
    
    for path in paths_to_add:
        if path not in sys.path:
            sys.path.insert(0, path)
            print(f"Added to path: {path}")
else:
    print("Could not find ModelChecker root directory. You may need to set paths manually.")

In [ ]:
# Import ModelChecker and check if it's working
try:
    # Reload if already imported to ensure we're using the correct version
    if "model_checker" in sys.modules:
        importlib.reload(sys.modules["model_checker"])
    import model_checker
    
    print(f"✓ ModelChecker version: {model_checker.__version__}")
    print(f"✓ Located at: {model_checker.__file__}")
    
    # Try importing the jupyter module specifically
    if "model_checker.jupyter" in sys.modules:
        importlib.reload(sys.modules["model_checker.jupyter"])
    from model_checker import jupyter
    print(f"✓ Jupyter module located at: {jupyter.__file__}")
    
except ImportError as e:
    print(f"✗ Error importing model_checker: {e}")
    print("\nCurrent sys.path:")
    for p in sys.path:
        print(f"  {p}")
    print("\nTroubleshooting tips:")
    print("1. Make sure you're running this notebook from the ModelChecker/Code directory")
    print("2. If using NixOS, make sure you've entered the nix-shell first")
    print("3. Try running the diagnostic script: python jupyter_test.py")

## 2. Basic Formula Checking

We can use the `check_formula` function to check the validity of a formula:

In [ ]:
# Import the check_formula function
try:
    from model_checker import check_formula
    
    # Check a simple formula using Unicode symbols
    result = check_formula("p → (q → p)")
    
    # The result is automatically displayed in the notebook
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Check a formula with premises
try:
    check_formula("q", premises=["p", "p → q"])
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

## 3. Formula Checking with Different Theories

We can check formulas using different semantic theories:

In [ ]:
# Get available theories
try:
    from model_checker.jupyter.environment import get_available_theories
    theories = get_available_theories()
    print(f"Available theories: {theories}")
except Exception as e:
    print(f"Error getting available theories: {e}")

In [ ]:
# Check a modal formula in the default theory
try:
    check_formula("□(p → q) → (□p → □q)", theory_name="default")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

## 4. Finding Countermodels

We can search for countermodels to invalid formulas:

In [ ]:
# Import the find_countermodel function
try:
    from model_checker import find_countermodel
    
    # Try to find a countermodel to an invalid formula
    find_countermodel("p → q", premises=[])
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

## 5. Interactive Model Explorer

For more interactive exploration, we can use the `ModelExplorer`:

In [ ]:
# Check if ipywidgets is available
try:
    import ipywidgets
    print(f"ipywidgets version: {ipywidgets.__version__}")
except ImportError:
    print("ipywidgets is not installed. Please install it using: pip install ipywidgets")
    print("Then run: jupyter nbextension enable --py widgetsnbextension")

In [ ]:
# Create and display an explorer
try:
    from model_checker import ModelExplorer
    
    explorer = ModelExplorer()
    explorer.display()
except Exception as e:
    print(f"Error creating ModelExplorer: {e}")
    import traceback
    traceback.print_exc()

## 6. Advanced Usage: Pre-configured Explorer

We can also create a pre-configured explorer for a specific formula:

In [ ]:
# Create a pre-configured explorer for a specific formula
try:
    from model_checker import explore_formula
    
    explore_formula("◇p ∧ ◇¬p", theory_name="default")
except Exception as e:
    print(f"Error creating pre-configured explorer: {e}")
    import traceback
    traceback.print_exc()

## 7. Unicode Operator Support

The integration supports both LaTeX and Unicode operators:

In [ ]:
try:
    from model_checker.jupyter.operators import unicode_to_latex, latex_to_unicode

    # Convert between Unicode and LaTeX notation
    unicode_formula = "p → (q ∧ ¬r) ∨ □s"
    latex_formula = unicode_to_latex(unicode_formula)

    print(f"Unicode: {unicode_formula}")
    print(f"LaTeX:   {latex_formula}")
    print(f"Back to Unicode: {latex_to_unicode(latex_formula)}")
except Exception as e:
    print(f"Error testing Unicode operators: {e}")
    import traceback
    traceback.print_exc()

## 8. Loading Examples from Theories

We can load and use pre-defined examples from theories:

In [ ]:
try:
    from model_checker.jupyter.utils import load_examples, get_example_categories

    # Load examples from the default theory
    examples = load_examples("default")
    if examples:
        # Group examples by category
        categories = get_example_categories(examples)
        
        # Print category summary
        for category, category_examples in categories.items():
            print(f"{category}: {len(category_examples)} examples")
            
        # Get the first example
        if examples:
            first_example_name = list(examples.keys())[0]
            first_example = examples[first_example_name]
            
            # Print example details
            print(f"\nExample '{first_example_name}'")
            print(f"Premises: {first_example[0]}")
            print(f"Conclusions: {first_example[1]}")
            print(f"Settings: {first_example[2]}")
    else:
        print("No examples found")
except Exception as e:
    print(f"Error loading examples: {e}")
    import traceback
    traceback.print_exc()

## 9. Using Examples from Theories

We can use predefined examples to check formulas:

In [ ]:
try:
    from model_checker import BuildExample, get_theory
    
    # Load a theory
    theory = get_theory("default")
    
    # Get examples from the theory (Modus Ponens is a common example)
    from model_checker.jupyter.utils import load_examples
    examples = load_examples("default")
    
    # Find a modus ponens example if one exists (may have a different name)
    example_name = None
    for name in examples.keys():
        if "MP" in name or "modus_ponens" in name.lower() or "modustoll" in name.lower():
            example_name = name
            break
            
    if example_name:
        print(f"Using example: {example_name}")
        # Create a model from the example
        model = BuildExample(example_name, theory)
        
        # Display the model
        from model_checker.jupyter.display import display_model
        display_model(model)
    else:
        print("Modus ponens example not found. Using a custom example instead.")
        
        # Create a custom modus ponens example
        example = [["p", "p → q"], ["q"], {'N': 3, 'max_time': 5}]
        model = BuildExample("modus_ponens", theory, example)
        
        # Display the model
        from model_checker.jupyter.display import display_model
        display_model(model)
        
except Exception as e:
    print(f"Error using example: {e}")
    import traceback
    traceback.print_exc()

## 10. Diagnostics (in case of issues)

If you're having trouble, this cell provides diagnostic information:

In [ ]:
try:
    from model_checker.jupyter.environment import get_diagnostic_info
    
    # Get diagnostic info
    diag_info = get_diagnostic_info()
    
    print("ModelChecker Diagnostics:")
    print(f"Python Version: {diag_info.get('python_version')}")
    
    # Model checker info
    mc_info = diag_info.get('model_checker', {})
    if isinstance(mc_info, dict):
        print(f"ModelChecker Path: {mc_info.get('path', 'Not found')}")
        print(f"ModelChecker Version: {mc_info.get('version', 'Unknown')}")
        print(f"Available Theories: {mc_info.get('theories', [])}")
    else:
        print(f"ModelChecker: {mc_info}")
    
    # Dependencies
    print("\nDependencies:")
    deps = diag_info.get('dependencies', {})
    for dep, version in deps.items():
        print(f"  {dep}: {version}")
    
    # Python path
    print("\nPython path:")
    for i, path in enumerate(diag_info.get('sys_path', [])):
        if i < 10:  # Limit to first 10 entries to avoid clutter
            print(f"  {path}")
except Exception as e:
    print(f"Error getting diagnostic info: {e}")
    
    # Fallback diagnostics
    print("\nBasic diagnostics:")
    print(f"Python version: {sys.version}")
    print(f"Working directory: {os.getcwd()}")
    try:
        import model_checker
        print(f"ModelChecker version: {model_checker.__version__}")
        print(f"ModelChecker path: {model_checker.__file__}")
    except ImportError:
        print("ModelChecker not importable")

## Conclusion

This notebook demonstrated the key features of the ModelChecker Jupyter integration. The modular architecture makes it easy to use and extend, with support for multiple theories, interactive exploration, and customizable visualizations.

For more detailed information, see the [Jupyter Integration Documentation](README.md).